### Housing prices dataset: by provinces and urban areas

In [2]:
import pandas as pd

# Load the spreadsheet
file_path = '../data/raw/Canada_Housing_Prices_2005_2024.xlsx'
spreadsheet = pd.ExcelFile(file_path)

# Display the names of the sheets
sheet_names = spreadsheet.sheet_names
sheet_names

['AGGREGATE',
 'BRITISH_COLUMBIA',
 'VANCOUVER_ISLAND',
 'VICTORIA',
 'LOWER_MAINLAND',
 'GREATER_VANCOUVER',
 'FRASER_VALLEY',
 'CHILLIWACK_AND_DISTRICT',
 'INTERIOR_BC',
 'ALBERTA',
 'CALGARY',
 'EDMONTON',
 'SASKATCHEWAN',
 'REGINA',
 'SASKATOON',
 'WINNIPEG',
 'ONTARIO',
 'BANCROFT_AND_AREA',
 'BARRIE_AND_DISTRICT',
 'BRANTFORD_REGION',
 'CAMBRIDGE',
 'GREY_BRUCE_OWEN_SOUND',
 'GUELPH_AND_DISTRICT',
 'HAMILTON_BURLINGTON',
 'HURON_PERTH',
 'KAWARTHA_LAKES',
 'KINGSTON_AND_AREA',
 'KITCHENER_WATERLOO',
 'LAKELANDS',
 'LONDON_ST_THOMAS',
 'MISSISSAUGA',
 'NIAGARA_REGION',
 'NORTH_BAY',
 'NORTHUMBERLAND_HILLS',
 'OAKVILLE_MILTON',
 'OTTAWA',
 'PETERBOROUGH_AND_KAWARTHAS',
 'QUINTE_AND_DISTRICT',
 'RIDEAU_ST_LAWRENCE',
 'SAULT_STE_MARIE',
 'SIMCOE_AND_DISTRICT',
 'SUDBURY',
 'TILLSONBURG_DISTRICT',
 'GREATER_TORONTO',
 'WINDSOR_ESSEX',
 'WOODSTOCK_INGERSOLL',
 'QUEBEC',
 'CENTRE_DU_QUEBEC',
 'ESTRIE',
 'MAURICIE',
 'MONTREAL_CMA',
 'QUEBEC_CMA',
 'NEW_BRUNSWICK',
 'FREDERICTON',
 'GREA

In [4]:
# Function to process each sheet: change date format and save to SQLite and CSV
import sqlite3

def process_and_save_sheet(sheet_name):
    # Read the sheet
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Change date column to Month-Year format
    df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m')
    
    # Save to CSV
    csv_path = f'../data/processed/housing_prices/{sheet_name}.csv'
    df.to_csv(csv_path, index=False)
    
    return df, csv_path

# Create an SQLite database and connection
db_path = '../data/sql/Canada_Housing_Prices_2005_2024.db'
conn = sqlite3.connect(db_path)

# Process and save each sheet
processed_sheets = {}
for sheet_name in sheet_names:
    df, csv_path = process_and_save_sheet(sheet_name)
    processed_sheets[sheet_name] = (df, csv_path)
    # Save dataframe to SQLite
    df.to_sql(sheet_name, conn, if_exists='replace', index=False)

conn.close()

processed_sheets.keys()


dict_keys(['AGGREGATE', 'BRITISH_COLUMBIA', 'VANCOUVER_ISLAND', 'VICTORIA', 'LOWER_MAINLAND', 'GREATER_VANCOUVER', 'FRASER_VALLEY', 'CHILLIWACK_AND_DISTRICT', 'INTERIOR_BC', 'ALBERTA', 'CALGARY', 'EDMONTON', 'SASKATCHEWAN', 'REGINA', 'SASKATOON', 'WINNIPEG', 'ONTARIO', 'BANCROFT_AND_AREA', 'BARRIE_AND_DISTRICT', 'BRANTFORD_REGION', 'CAMBRIDGE', 'GREY_BRUCE_OWEN_SOUND', 'GUELPH_AND_DISTRICT', 'HAMILTON_BURLINGTON', 'HURON_PERTH', 'KAWARTHA_LAKES', 'KINGSTON_AND_AREA', 'KITCHENER_WATERLOO', 'LAKELANDS', 'LONDON_ST_THOMAS', 'MISSISSAUGA', 'NIAGARA_REGION', 'NORTH_BAY', 'NORTHUMBERLAND_HILLS', 'OAKVILLE_MILTON', 'OTTAWA', 'PETERBOROUGH_AND_KAWARTHAS', 'QUINTE_AND_DISTRICT', 'RIDEAU_ST_LAWRENCE', 'SAULT_STE_MARIE', 'SIMCOE_AND_DISTRICT', 'SUDBURY', 'TILLSONBURG_DISTRICT', 'GREATER_TORONTO', 'WINDSOR_ESSEX', 'WOODSTOCK_INGERSOLL', 'QUEBEC', 'CENTRE_DU_QUEBEC', 'ESTRIE', 'MAURICIE', 'MONTREAL_CMA', 'QUEBEC_CMA', 'NEW_BRUNSWICK', 'FREDERICTON', 'GREATER_MONCTON', 'SAINT_JOHN_NB', 'NOVA_SCOTIA'

In [3]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
db_path = '../data/sql/Canada_Housing_Prices_2005_2024.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Fetch all table names in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
all_tables = [row[0] for row in cursor.fetchall()]

# Filter out the table you want to skip
tables_to_iterate = [table for table in all_tables if table != 'Housing_Price_Index']

# Initialize an empty DataFrame for the merged data
merged_df = pd.DataFrame()

# Iterate through the remaining tables
for i, table_name in enumerate(tables_to_iterate):
    # Load the table
    df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    
    # If not the first table, drop the 'Date' column (or any other common columns) to avoid duplicates
    if i > 0:
        df = df.drop(columns=['Date'])
    
    # Merge with the cumulative DataFrame
    merged_df = pd.concat([merged_df, df], axis=1)


In [6]:
merged_df.head()

,Month-Year,Overnight money market financing,Composite_HPI_SA,Single_Family_HPI_SA,One_Storey_HPI_SA,Two_Storey_HPI_SA,Townhouse_HPI_SA,Apartment_HPI_SA,Composite_Benchmark_SA,Single_Family_Benchmark_SA,...,One_Storey_HPI_SA,Two_Storey_HPI_SA,Townhouse_HPI_SA,Apartment_HPI_SA,Composite_Benchmark_SA,Single_Family_Benchmark_SA,One_Storey_Benchmark_SA,Two_Storey_Benchmark_SA,Townhouse_Benchmark_SA,Apartment_Benchmark_SA
0,1980-01,13.27,100.0,100.0,100.0,100.0,100.0,100.0,239800.0,260400.0,...,100.0,100.0,100.0,100.0,239800,260400,207700,302900,201700.0,172000.0
1,1980-02,11.98,100.3,100.2,100.3,100.1,100.3,100.6,240500.0,261000.0,...,100.3,100.1,100.3,100.6,240500,261000,208400,303300,202300.0,173000.0
2,1980-03,11.43,100.6,100.6,100.7,100.5,100.6,101.1,241300.0,261900.0,...,100.7,100.5,100.6,101.1,241300,261900,209200,304300,202900.0,173900.0
3,1980-04,15.84,100.9,100.8,101.2,100.6,100.8,101.5,242000.0,262600.0,...,101.2,100.6,100.8,101.5,242000,262600,210100,304800,203300.0,174600.0
4,1980-05,17.05,101.2,101.1,101.4,100.8,101.0,102.0,242600.0,263200.0,...,101.4,100.8,101.0,102.0,242600,263200,210600,305400,203700.0,175400.0


In [8]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Fetch all table names in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
all_tables = [row[0] for row in cursor.fetchall()]

# Iterate through all tables and rename 'Month-Year' (or similar) to 'Date'
for table in all_tables:
    # Fetch the columns in the table
    cursor.execute(f"PRAGMA table_info({table});")
    columns_info = cursor.fetchall()
    
    # Check if 'Month-Year' or similar exists
    for column in columns_info:
        column_name = column[1]
        if 'Month' in column_name or 'Year' in column_name:
            # Rename the column to 'Date'
            cursor.execute(f"""
            ALTER TABLE {table}
            RENAME COLUMN '{column_name}' TO 'Date';
            """)
            print(f"Renamed column '{column_name}' to 'Date' in table '{table}'.")

# Commit the changes and close the connection
conn.commit()
conn.close()


Renamed column 'Month-Year' to 'Date' in table 'Bank_of_Canada_Interest_Rate'.
Renamed column 'Month-Year' to 'Date' in table 'Housing_Price_Index'.


In [13]:
merged_df

,Date,Overnight money market financing,Composite_HPI_SA,Single_Family_HPI_SA,One_Storey_HPI_SA,Two_Storey_HPI_SA,Townhouse_HPI_SA,Apartment_HPI_SA,Composite_Benchmark_SA,Single_Family_Benchmark_SA,...,One_Storey_HPI_SA,Two_Storey_HPI_SA,Townhouse_HPI_SA,Apartment_HPI_SA,Composite_Benchmark_SA,Single_Family_Benchmark_SA,One_Storey_Benchmark_SA,Two_Storey_Benchmark_SA,Townhouse_Benchmark_SA,Apartment_Benchmark_SA
0,1980-01,13.27,100.0,100.0,100.0,100.0,100.0,100.0,239800.0,260400.0,...,100.0,100.0,100.0,100.0,239800,260400,207700,302900,201700.0,172000.0
1,1980-02,11.98,100.3,100.2,100.3,100.1,100.3,100.6,240500.0,261000.0,...,100.3,100.1,100.3,100.6,240500,261000,208400,303300,202300.0,173000.0
2,1980-03,11.43,100.6,100.6,100.7,100.5,100.6,101.1,241300.0,261900.0,...,100.7,100.5,100.6,101.1,241300,261900,209200,304300,202900.0,173900.0
3,1980-04,15.84,100.9,100.8,101.2,100.6,100.8,101.5,242000.0,262600.0,...,101.2,100.6,100.8,101.5,242000,262600,210100,304800,203300.0,174600.0
4,1980-05,17.05,101.2,101.1,101.4,100.8,101.0,102.0,242600.0,263200.0,...,101.4,100.8,101.0,102.0,242600,263200,210600,305400,203700.0,175400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,241.7,228.6,216.5,222.3,333800,346300,325100,368900,280100.0,236800.0
14270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,250.5,230.9,219.9,223.6,341300,354600,336900,372700,284600.0,238100.0
14271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,250.7,233.0,220.3,218.3,342900,356600,337200,376100,285100.0,232500.0
14272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,249.5,234.4,225.9,214.6,342500,356900,335600,378300,292300.0,228500.0


In [14]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Load the AGGREGATE table to determine the date range
aggregate_df = pd.read_sql("SELECT Date FROM AGGREGATE", conn)
aggregate_df['Date'] = pd.to_datetime(aggregate_df['Date'], format='%Y-%m')

# Find the minimum and maximum dates in the AGGREGATE table
min_date = aggregate_df['Date'].min()
max_date = aggregate_df['Date'].max()

# Load the Bank_of_Canada_Interest_Rate table
interest_rate_df = pd.read_sql("SELECT * FROM Bank_of_Canada_Interest_Rate", conn)
interest_rate_df['Date'] = pd.to_datetime(interest_rate_df['Date'], format='%Y-%m')

# Filter the interest rate data to only include rows within the date range
filtered_interest_rate_df = interest_rate_df[
    (interest_rate_df['Date'] >= min_date) & 
    (interest_rate_df['Date'] <= max_date)
]



In [15]:
filtered_interest_rate_df

,Date,Overnight money market financing
300,2005-01-01,2.50
301,2005-02-01,2.50
302,2005-03-01,2.49
303,2005-04-01,2.49
304,2005-05-01,2.49
...,...,...
529,2024-02-01,5.02
530,2024-03-01,5.00
531,2024-04-01,5.00
532,2024-05-01,5.00


In [18]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Rename the column in the Bank_of_Canada_Interest_Rate table
cursor.execute("""
ALTER TABLE Bank_of_Canada_Interest_Rate
RENAME COLUMN 'Overnight money market financing' TO 'Interest Rate';
""")

# Commit the changes and close the connection
conn.commit()
conn.close()


In [24]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Load the AGGREGATE table to determine the date range
aggregate_df = pd.read_sql("SELECT * FROM AGGREGATE", conn)
aggregate_df['Date'] = pd.to_datetime(aggregate_df['Date'], format='%Y-%m')

# Load the Bank_of_Canada_Interest_Rate table
interest_rate_df = pd.read_sql("SELECT * FROM Bank_of_Canada_Interest_Rate", conn)
interest_rate_df['Date'] = pd.to_datetime(interest_rate_df['Date'], format='%Y-%m')

# Merge the datasets on Date
merged_df = pd.merge(aggregate_df, interest_rate_df, on='Date', how='left')

# Format the Date column to "YYYY-MM"
merged_df['Date'] = merged_df['Date'].dt.strftime('%Y-%m')

# Save the corrected merged DataFrame back to CSV
merged_csv_path = '../data/processed/Merged_Canada_HPI_Interet_Rate.csv'
merged_df.to_csv(merged_csv_path, index=False)

# Close the connection
conn.close()

merged_csv_path

'../data/processed/Merged_Canada_HPI_Interet_Rate.csv'

In [25]:
merged_df

,Date,Composite_HPI_SA,Single_Family_HPI_SA,One_Storey_HPI_SA,Two_Storey_HPI_SA,Townhouse_HPI_SA,Apartment_HPI_SA,Composite_Benchmark_SA,Single_Family_Benchmark_SA,One_Storey_Benchmark_SA,Two_Storey_Benchmark_SA,Townhouse_Benchmark_SA,Apartment_Benchmark_SA,Interest Rate
0,2005-01,100.0,100.0,100.0,100.0,100.0,100.0,239800,260400,207700,302900,201700,172000,2.50
1,2005-02,100.3,100.2,100.3,100.1,100.3,100.6,240500,261000,208400,303300,202300,173000,2.50
2,2005-03,100.6,100.6,100.7,100.5,100.6,101.1,241300,261900,209200,304300,202900,173900,2.49
3,2005-04,100.9,100.8,101.2,100.6,100.8,101.5,242000,262600,210100,304800,203300,174600,2.49
4,2005-05,101.2,101.1,101.4,100.8,101.0,102.0,242600,263200,210600,305400,203700,175400,2.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,2024-02,300.3,304.6,294.8,307.0,328.8,306.6,720000,793200,612200,929900,663100,527400,5.02
230,2024-03,299.5,304.9,295.1,306.7,329.1,305.5,718300,794000,612900,929000,663700,525400,5.00
231,2024-04,299.5,304.7,295.8,306.9,328.2,304.2,718300,793500,614400,929700,662000,523200,5.00
232,2024-05,298.9,304.2,295.7,306.4,327.9,303.3,716800,792200,614100,928200,661300,521700,5.00
